# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [2]:
import pandas as pd

# Load the dataset
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

# Standardize column names for easier access
df.columns = df.columns.str.lower().str.replace(" ", "_").str.strip()

# Inspect the dataset
print("Dataset Overview:")
display(df.head())

Dataset Overview:


,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


In [4]:
# Create a filtered DataFrame
filtered_df = df[(df['total_claim_amount'] > 1000) & (df['response'] == 'Yes')]

# Display the new DataFrame
print("\nFiltered DataFrame (Total Claim > $1,000 and Response = 'Yes'):")
display(filtered_df)


Filtered DataFrame (Total Claim > $1,000 and Response = 'Yes'):


,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
189,189,OK31456,California,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,NaN
236,236,YJ16163,Oregon,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Special Auto,Special L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,A
419,419,GW43195,Oregon,25807.063000,Yes,Extended,College,2/13/11,Employed,F,...,1.0,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,Luxury Car,Small,A
442,442,IP94270,Arizona,13736.132500,Yes,Premium,Master,2/13/11,Disabled,F,...,0.0,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,SUV,Medsize,A
587,587,FJ28407,California,5619.689084,Yes,Premium,High School or Below,1/26/11,Unemployed,M,...,0.0,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,SUV,Medsize,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10351,10351,FN44127,Oregon,3508.569533,Yes,Extended,College,1/5/11,Medical Leave,M,...,1.0,1,Personal Auto,Personal L2,Offer2,Branch,1176.278800,Four-Door Car,Small,NaN
10373,10373,XZ64172,Oregon,10963.957230,Yes,Premium,High School or Below,2/8/11,Employed,M,...,0.0,1,Corporate Auto,Corporate L2,Offer1,Agent,1324.800000,Luxury SUV,Medsize,NaN
10487,10487,IX60941,Oregon,3508.569533,Yes,Extended,College,1/5/11,Medical Leave,M,...,1.0,1,Personal Auto,Personal L3,Offer2,Branch,1176.278800,Four-Door Car,Small,NaN
10565,10565,QO62792,Oregon,7840.165778,Yes,Extended,College,1/14/11,Employed,M,...,2.0,1,Personal Auto,Personal L3,Offer2,Agent,1008.000000,NaN,NaN,NaN


In [5]:
# Filter for customers who responded "Yes"
responded_yes_df = df[df['response'] == 'Yes']

# Group by policy type and gender to calculate the average total claim amount
avg_claim = responded_yes_df.groupby(['policy_type', 'gender'])['total_claim_amount'].mean().reset_index()

# Display the result
print("\nAverage Total Claim Amount by Policy Type and Gender (Response = 'Yes'):")
print(avg_claim)


Average Total Claim Amount by Policy Type and Gender (Response = 'Yes'):
      policy_type gender  total_claim_amount
0  Corporate Auto      F          433.738499
1  Corporate Auto      M          408.582459
2   Personal Auto      F          452.965929
3   Personal Auto      M          457.010178
4    Special Auto      F          453.280164
5    Special Auto      M          429.527942


In [6]:
# Count the total number of customers by state
state_counts = df['state'].value_counts().reset_index()
state_counts.columns = ['state', 'customer_count']

# Filter states with more than 500 customers
states_with_500_plus = state_counts[state_counts['customer_count'] > 500]

# Display the filtered results
print("\nStates with More Than 500 Customers:")
print(states_with_500_plus)


States with More Than 500 Customers:
        state  customer_count
0  California            3552
1      Oregon            2909
2     Arizona            1937
3      Nevada             993
4  Washington             888


In [7]:
# Group by education and gender to calculate max, min, and median CLV
clv_stats = df.groupby(['education', 'gender'])['customer_lifetime_value'].agg(['max', 'min', 'median']).reset_index()

# Display the results
print("\nCustomer Lifetime Value Statistics by Education and Gender:")
print(clv_stats)


Customer Lifetime Value Statistics by Education and Gender:
              education gender          max          min       median
0              Bachelor      F  73225.95652  1904.000852  5640.505303
1              Bachelor      M  67907.27050  1898.007675  5548.031892
2               College      F  61850.18803  1898.683686  5623.611187
3               College      M  61134.68307  1918.119700  6005.847375
4                Doctor      F  44856.11397  2395.570000  5332.462694
5                Doctor      M  32677.34284  2267.604038  5577.669457
6  High School or Below      F  55277.44589  2144.921535  6039.553187
7  High School or Below      M  83325.38119  1940.981221  6286.731006
8                Master      F  51016.06704  2417.777032  5729.855012
9                Master      M  50568.25912  2272.307310  5579.099207


## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [8]:
# Standardize column names
df.columns = df.columns.str.lower().str.replace(" ", "_").str.strip()

# Simulate a 'date_of_policy' column for example purposes
df['date_of_policy'] = pd.date_range(start='2023-01-01', periods=len(df), freq='D')

# Extract the month name
df['month'] = df['date_of_policy'].dt.month_name()


In [9]:
# Group by state and month and count the policies
policies_by_state_month = df.groupby(['state', 'month']).size().reset_index(name='number_of_policies')

# Sort the data by the number of policies sold in descending order
policies_by_state_month = policies_by_state_month.sort_values(by='number_of_policies', ascending=False)


In [11]:
# Calculate total policies sold per state
total_policies_by_state = policies_by_state_month.groupby('state')['number_of_policies'].sum().sort_values(ascending=False)

# Get the top 3 states
top_3_states = total_policies_by_state.head(3).index

In [12]:
# Filter for top 3 states
top_3_states_data = policies_by_state_month[policies_by_state_month['state'].isin(top_3_states)]

# Create a pivot table with months as columns and states as rows
top_3_pivot = top_3_states_data.pivot(index='state', columns='month', values='number_of_policies').fillna(0)

# Display the pivot table
print("\nNumber of Policies Sold by Month (Top 3 States):")
print(top_3_pivot)


Number of Policies Sold by Month (Top 3 States):
month       April  August  December  February  January  July  June  March  \
state                                                                       
Arizona       157     180       146       155      147   174   161    160   
California    281     280       295       275      302   297   304    316   
Oregon        237     236       258       225      266   240   236    248   

month       May  November  October  September  
state                                          
Arizona     165       174      165        153  
California  301       285      316        300  
Oregon      244       214      246        259  


In [13]:
# Melt the data to focus on marketing channel and response
melted_df = df.melt(
    id_vars=['response'],
    value_vars=['sales_channel'],
    var_name='marketing_channel',
    value_name='channel'
)

In [14]:
# Calculate the response rate by channel
response_rate = melted_df.groupby('channel')['response'].apply(
    lambda x: (x == 'Yes').mean() * 100
).reset_index(name='response_rate')

# Display the response rate
print("\nCustomer Response Rate by Marketing Channel:")
print(response_rate)


Customer Response Rate by Marketing Channel:
       channel  response_rate
0        Agent      18.005339
1       Branch      10.787558
2  Call Center      10.322279
3          Web      10.885609
